In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import time

from selenium import webdriver

In [3]:
years = list(range(1991, 2021))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

# Scrape MVPs

In [4]:
dfs = []

# Already fetched
# for year in years:
#   url = url_start.format(year)
#   data = requests.get(url)
#   file_name = "./mvps/{}.html".format(year)


#   if os.path.exists(file_name):
#     os.remove(file_name)
#   with open(file_name, "w+") as f:
#     f.write(data.text)
  

In [5]:
# for year in years:
#   with open("./mvps/{}.html".format(year)) as f:
#     page = f.read()
  
#   soup = bs(page, "html.parser")
#   soup.find("tr", class_="over_header").decompose()
#   mvp_table = soup.find_all(id="mvp")
#   mvp = pd.read_html(str(mvp_table))[0]
#   mvp["Year"] = year

#   print(year)

#   dfs.append(mvp)
#   mvps = pd.concat(dfs)


In [6]:
# if os.path.exists("mvps.csv"):
#   os.remove("mvps.csv")
# mvps.to_csv("mvps.csv")

# Scrape player stats

In [7]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [8]:
# driver = webdriver.Remote(
#   command_executor = os.environ["SELENIUM_URL"],
#   options = webdriver.ChromeOptions()
# )

# for year in years:
#     url = player_stats_url.format(year)
#     print(f"scraping {year} data")
    
#     driver.get(url)
#     driver.execute_script("window.scrollTo(1,10000)")
#     time.sleep(2)
    
#     with open("player/{}.html".format(year), "w+") as f:
#         f.write(driver.page_source)

# driver.close()

scraping 2013 data
scraping 2014 data
scraping 2015 data
scraping 2016 data
scraping 2017 data
scraping 2018 data
scraping 2019 data


In [9]:
dfs = []

for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()
    
    soup = bs(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

players = pd.concat(dfs)
players.to_csv("players.csv")


/tmp/ipykernel_5534/1397376143.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
/tmp/ipykernel_5534/1397376143.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
/tmp/ipykernel_5534/1397376143.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
/tmp/ipykernel_5534/1397376143.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_htm